# Feature engineering

> In this module, we develop tools to extract features from compounds, proteins, etc.

In [ ]:
#| default_exp feature

In [ ]:
#| hide
import sys
sys.path.append("/notebooks/katlas")
from nbdev.showdoc import *
%matplotlib inline

In [ ]:
#| export
from katlas.core import Data
import seaborn as sns
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.ML.Descriptors import MoleculeDescriptors
import pandas as pd
from rdkit.Chem import Draw
from rdkit.Chem import Descriptors
from sklearn.preprocessing import StandardScaler

## Features from amino acid

### RDKit descriptors

In [ ]:
#| export
from fastbook import *
from fairscale.nn.data_parallel import FullyShardedDataParallel as FSDP
from fairscale.nn.wrap import enable_wrap, wrap
import esm
from tqdm.notebook import tqdm; tqdm.pandas()
import gc

In [ ]:
#| export
def smi2prop(df, # df needs to have SMILES an ID columns
             smi_colname = "SMILES", # column name of smiles
             id_colname = "ID", # column name of ID
             remove_duplicate=False, # remove features that are highly correlated
             thr = 0.95, # threshold of Pearson correlation
             normalize = True, # normalize features using StandardScaler()
            ):
    "Extract ~209 features from smiles via rdkit.Chem.Descriptors, and remove duplicate features"
    
    mols = [Chem.MolFromSmiles(smi) for smi in df[smi_colname]]
    desc_names = [desc_name[0] for desc_name in Descriptors.descList]
    desc_calc = MoleculeDescriptors.MolecularDescriptorCalculator(desc_names)
    desc_values = [desc_calc.CalcDescriptors(mol) for mol in mols]
    feature_df = pd.DataFrame(np.stack(desc_values), index=df[id_colname],columns=desc_names)
    if remove_duplicate:
        # remove compound that has same value across features
        # feature_df = feature_df.loc[feature_df.std(axis=1) != 0] 
        print(f'number of {feature_df.shape[1]} features are detected')
        #femove features with zero std
        feature_std = feature_df.std()
        zero_std_features = np.where(feature_std == 0)[0]
        to_drop = feature_df.columns[zero_std_features]
        feature_df = feature_df.drop(columns=to_drop).copy()
        print(f'dropping {len(to_drop)} features, as they have zero std variance:{to_drop.tolist()}')
        
        corr_matrix = feature_df.corr().abs()
        # Select upper triangle of correlation matrix
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        # Find index of feature columns with correlation greater than a threshold (e.g., 0.95)
        to_drop = [column for column in upper.columns if any(upper[column] > thr)]
        # Drop the highly correlated features 
        feature_df = feature_df.drop(to_drop, axis=1).copy()
        print(f'dropping {len(to_drop)} features, as they have Pearson corr > {thr}:{to_drop}')
        print(f'number of {feature_df.shape[1]} features are left')
        

    if normalize:
        scaler = StandardScaler()
        transformed = scaler.fit_transform(feature_df.iloc[:,1:])
        feature_df.iloc[:,1:] = transformed
        
    feature_df = feature_df.reset_index()
    return feature_df

In [ ]:
show_doc(smi2prop)

In [ ]:
df = Data.get_aa_info()[['aa','SMILES']]

In [ ]:
df_feature = smi2prop(df, id_colname='aa',remove_duplicate=True,thr=0.9)

In [ ]:
df_feature

In [ ]:
#df_feature.to_csv('aa_feature.csv',index=False)

### Morgan fingerprint

In [ ]:
#| export
def smi2morgan(df, # a dataframe contains ID and SMILES columns
               smi_colname = "SMILES", # set smiles columne name
               id_colname = "ID", # set ID column name
              ):
    "Like `smi2prop`, get 2048 morgan feature (0/1) given a dataframe that contains ID&smiles"
    mols = [Chem.MolFromSmiles(smi) for smi in df[smi_colname]]
    morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048) for mol in mols]
    fp_df = pd.DataFrame(np.array(morgan_fps), index=df[id_colname])
    colnames = [f'morgan_{i}' for i in fp_df.columns]
    fp_df.columns = colnames
    fp_df = fp_df.reset_index()
    return fp_df

In [ ]:
show_doc(smi2morgan)

## Features from protein sequence

In [ ]:
#| export
def esm_embeddings(df: pd.DataFrame, 
                   seq_colname: str, #The name of the column containing the sequences.
                   model_name: str = "esm2_t33_650M_UR50D", #The name of the ESM model to use for the embeddings.
                  ) -> pd.DataFrame:
    """
    Extract 1280 esmfold2 embeddings from protein sequence.
    """
    
    # Initialize distributed world with world_size 1
    if not torch.distributed.is_initialized():
        url = "tcp://localhost:23456"
        torch.distributed.init_process_group(backend="nccl", init_method=url, world_size=1, rank=0)
    
    #get number of repr layers
    match = re.search(r'_t(\d+)_', model_name)
    number = int(match.group(1))
    print(f"repr_layers number for model {model_name} is {number}.")
    print("You can also choose other esm2 models:",
          "\nesm2_t48_15B_UR50D\nesm2_t36_3B_UR50D\nesm2_t33_650M_UR50D\nesm2_t30_150M_UR50D\nesm2_t12_35M_UR50D\nesm2_t6_8M_UR50D\n")

    # Download model data from the hub
    model_data, regression_data = esm.pretrained._download_model_and_regression_data(model_name)

    # Initialize the model with FSDP wrapper
    fsdp_params = dict(
        mixed_precision=True,
        flatten_parameters=True,
        state_dict_device=torch.device("cpu"),  # reduce GPU mem usage
        cpu_offload=True,  # enable cpu offloading
    )

    with enable_wrap(wrapper_cls=FSDP, **fsdp_params):
        model, vocab = esm.pretrained.load_model_and_alphabet_core(
            model_name, model_data, regression_data
        )
        batch_converter = vocab.get_batch_converter()
        model.eval()

        # Wrap each layer in FSDP separately
        for name, child in model.named_children():
            if name == "layers":
                for layer_name, layer in child.named_children():
                    wrapped_layer = wrap(layer)
                    setattr(child, layer_name, wrapped_layer)
        model = wrap(model)

        # Define the feature extraction function
        def get_feature(r, colname=seq_colname) -> np.ndarray:
            data = [('protein', r[colname])]
            labels, strs, tokens = batch_converter(data)
            with torch.no_grad():
                results = model(tokens.cuda(), repr_layers=[number], return_contacts=False)
            rpr = results["representations"][number].squeeze()
            rpr = rpr[1 : len(r[colname]) + 1].mean(0).detach().cpu().numpy()

            del results, labels, strs, tokens, data #especially need to delete those on cuda: tokens, results
            gc.collect()

            return rpr
        
        # Apply the feature extraction function to each row in the DataFrame
        series = df.progress_apply(get_feature, axis=1)
        df_feature = pd.DataFrame(series.tolist())

        return df_feature

In [ ]:
show_doc(esm_embeddings)

In [ ]:
df = Data.get_esm_pca32()

In [ ]:
df.shape

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()